In [1]:
"""
Merge all csv files from the Canadian Institute of Cybersecurity dataset available on AWS Public Data.
"""

'\nMerge all csv files from the Canadian Institute of Cybersecurity dataset available on AWS Public Data.\n'

In [2]:
"""
Import libraries.
"""
import pandas as pd
import numpy as np
import boto3
import math
import json
import os
from IPython.display import Image
from IPython.display import display
from sklearn.datasets import dump_svmlight_file
from time import gmtime, strftime
import sys
import sagemaker

In [5]:
"""
Set up AWS resources.
"""
role = sagemaker.get_execution_role()
s3=boto3.resource('s3')
# Note: All cic data was previously copied to my own bucket, cic-network-data via aws cli.
bucket = 'cic-network-data'

f1='Friday-02-03-2018_TrafficForML_CICFlowMeter.csv'
f2='Friday-16-02-2018_TrafficForML_CICFlowMeter.csv'
f3='Friday-23-02-2018_TrafficForML_CICFlowMeter.csv'
f4='Thuesday-20-02-2018_TrafficForML_CICFlowMeter.csv'
f5='Thursday-01-03-2018_TrafficForML_CICFlowMeter.csv'
f6='Thursday-15-02-2018_TrafficForML_CICFlowMeter.csv'
f7='Thursday-22-02-2018_TrafficForML_CICFlowMeter.csv'
f8='Wednesday-14-02-2018_TrafficForML_CICFlowMeter.csv'
f9='Wednesday-21-02-2018_TrafficForML_CICFlowMeter.csv'
f10='Wednesday-28-02-2018_TrafficForML_CICFlowMeter.csv'


In [6]:
""" 
Download files to notebook instance.
"""
s3.Bucket(bucket).download_file(f1, 'f1.csv')
s3.Bucket(bucket).download_file(f2, 'f2.csv')
s3.Bucket(bucket).download_file(f3, 'f3.csv')
s3.Bucket(bucket).download_file(f4, 'f4.csv')
s3.Bucket(bucket).download_file(f5, 'f5.csv')
s3.Bucket(bucket).download_file(f6, 'f6.csv')
s3.Bucket(bucket).download_file(f7, 'f7.csv')
s3.Bucket(bucket).download_file(f8, 'f8.csv')
s3.Bucket(bucket).download_file(f9, 'f9.csv')
s3.Bucket(bucket).download_file(f10, 'f10.csv')

In [8]:
"""
Load in all files to pandas to merge.
"""
df1 = pd.read_csv('./f1.csv')
df2 = pd.read_csv('./f2.csv')
df3 = pd.read_csv('./f3.csv')
df4 = pd.read_csv('./f4.csv')
df5 = pd.read_csv('./f5.csv')
df6 = pd.read_csv('./f6.csv')
df7 = pd.read_csv('./f7.csv')
df8 = pd.read_csv('./f8.csv')
df9 = pd.read_csv('./f9.csv')
df10 = pd.read_csv('./f10.csv')
pd.set_option('display.max_rows', 20)

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (0,1,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivi

In [9]:
"""
Check to make sure the schemas match up before merging.
"""
print(df1.shape, df2.shape, df3.shape, df4.shape, df5.shape, \
      df6.shape, df7.shape, df8.shape, df9.shape, df10.shape)

(1048575, 80) (1048575, 80) (1048575, 80) (7948748, 84) (331125, 80) (1048575, 80) (1048575, 80) (1048575, 80) (1048575, 80) (613104, 80)


In [11]:
"""
df4 does not match our schema. Let's fix that.
"""
df1.head()

,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,443,6,02/03/2018 08:47:38,141385,9,7,553,3773.0,202,0,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign
1,49684,6,02/03/2018 08:47:38,281,2,1,38,0.0,38,0,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign
2,443,6,02/03/2018 08:47:40,279824,11,15,1086,10527.0,385,0,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign
3,443,6,02/03/2018 08:47:40,132,2,0,0,0.0,0,0,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign
4,443,6,02/03/2018 08:47:41,274016,9,13,1285,6141.0,517,0,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign


In [12]:
df4.head()

,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,172.31.69.25-94.231.103.172-22-45498-6,94.231.103.172,45498,172.31.69.25,22,6,20/02/2018 08:34:07,888751,11,11,...,32,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,Benign
1,8.0.6.4-8.6.0.1-0-0-0,8.6.0.1,0,8.0.6.4,0,0,20/02/2018 08:33:22,112642816,3,0,...,0,0.0,0.0,0.0,0.0,56300000.0,7.071068,56300000.0,56300000.0,Benign
2,8.0.6.4-8.6.0.1-0-0-0,8.6.0.1,0,8.0.6.4,0,0,20/02/2018 08:36:11,112642712,3,0,...,0,0.0,0.0,0.0,0.0,56300000.0,18.384776,56300000.0,56300000.0,Benign
3,8.0.6.4-8.6.0.1-0-0-0,8.6.0.1,0,8.0.6.4,0,0,20/02/2018 08:39:00,112642648,3,0,...,0,0.0,0.0,0.0,0.0,56300000.0,5.656854,56300000.0,56300000.0,Benign
4,8.0.6.4-8.6.0.1-0-0-0,8.6.0.1,0,8.0.6.4,0,0,20/02/2018 08:41:49,112642702,3,0,...,0,0.0,0.0,0.0,0.0,56300000.0,65.053824,56300000.0,56300000.0,Benign


In [13]:
"""
We need to drop Flow ID, Src IP, Src Port, Dst IP from df4.
"""
df4.drop(['Flow ID', 'Src IP', 'Src Port', 'Dst IP'], axis=1, inplace=True)
df4.shape

(7948748, 80)

In [14]:
"""
Merge dataframes and make sure the new shape makes sense.
"""
merged_df = pd.concat([df1, df2, df3, df4, df5, df6, df7, df8, df9, df10])
merged_df.shape

(16233002, 80)

In [15]:
merged_df.head()

,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,443,6,02/03/2018 08:47:38,141385,9,7,553,3773,202,0,...,20,0,0,0,0,0,0,0,0,Benign
1,49684,6,02/03/2018 08:47:38,281,2,1,38,0,38,0,...,20,0,0,0,0,0,0,0,0,Benign
2,443,6,02/03/2018 08:47:40,279824,11,15,1086,10527,385,0,...,20,0,0,0,0,0,0,0,0,Benign
3,443,6,02/03/2018 08:47:40,132,2,0,0,0,0,0,...,20,0,0,0,0,0,0,0,0,Benign
4,443,6,02/03/2018 08:47:41,274016,9,13,1285,6141,517,0,...,20,0,0,0,0,0,0,0,0,Benign


In [16]:
merged_df.tail()

,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
613099,23,6,28/02/2018 11:59:12,3,1,1,0,0,0,0,...,24,0,0,0,0,0,0,0,0,Infilteration
613100,425,6,28/02/2018 10:50:04,2,1,1,0,0,0,0,...,24,0,0,0,0,0,0,0,0,Infilteration
613101,445,6,28/02/2018 12:52:55,732728,2,2,0,0,0,0,...,32,0,0,0,0,0,0,0,0,Benign
613102,23,6,28/02/2018 11:10:50,22,1,1,0,0,0,0,...,24,0,0,0,0,0,0,0,0,Infilteration
613103,443,6,28/02/2018 11:12:18,2,1,1,0,0,0,0,...,24,0,0,0,0,0,0,0,0,Infilteration


In [17]:
"""
Save base dataframe to our notebook instance as input to our clean function.
"""
merged_df.to_csv('base_df.csv', index=False)

In [19]:
"""
Save base dataframe to our S3 bucket.
"""
s3.Bucket(bucket).upload_file('base_df.csv', 'base_df.csv')